In [1]:
import re
import numpy as np
import pandas as pd
import os

In [2]:
# Importing libraries which will assist in removing the html tags
from io import StringIO
from html.parser import HTMLParser

"""Remove html tags from messages"""

class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [3]:
directory = 'txt_files/Raw'
faq_dict = {}
counter = 0

for filename in os.listdir(directory):
    if (filename.endswith(".txt") and 'blog_' not in filename):
        f = open('txt_files/Raw/{}'.format(filename), 'r+')
        string_list = f.readlines()
        
        if len(string_list) > 0:
            
            # Cleans blank lines and random sub-headings from the text (Part 1 of 2)
            new_lines = [line for line in string_list if line != '\n' and line != ' \n']
            new_lines = [strip_tags(line) for line in new_lines]
            new_lines = [re.sub(' +', ' ', line) for line in new_lines]
            new_lines = [re.sub('\xa0', '', line) for line in new_lines]
            for i in range(len(new_lines) - 1):
                if new_lines[i] == new_lines[i+1]:
                    new_lines[i] = ' '
            for i in range(len(new_lines)):
                if new_lines[i].startswith(' '):
                    new_lines[i] = ' '
            new_lines = [line for line in new_lines if line != ' ']

            try:
                # Look for questions that aren't in headings
                for i in range(len(new_lines)):
                    if ('?' in new_lines[i] and 'H2' not in new_lines[i] and 'H3' not in new_lines[i]):
                        if new_lines[i] in faq_dict.keys():
                            a = faq_dict[new_lines[i]]
                            if new_lines[i + 1] not in a:
                                a.append(new_lines[i + 1])
                                faq_dict[new_lines[i]] = a
                        else:
                            faq_dict[new_lines[i]] = [new_lines[i + 1]]

                # Look for questions in H2 headings
                    if ('?' in new_lines[i] and 'H2' in new_lines[i]):
                        key = new_lines[i]
                        vals = []
                        j = i + 1
                        while 'H2' not in new_lines[j]:
                            vals.append(new_lines[j])
                            j += 1
                        if key in faq_dict.keys():
                            a = faq_dict[key]
                            for k in range(len(vals)):
                                if vals[k] not in a:
                                    a.append(vals[k])
                                    faq_dict[key] = a
                        else:
                            faq_dict[key] = vals

                # Look for questions in H3 headings
                    if ('?' in new_lines[i] and 'H3' in new_lines[i]):
                        key = new_lines[i]
                        vals = []
                        j = i + 1
                        while 'H2' not in new_lines[j]:
                            vals.append(new_lines[j])
                            j += 1
                        if key in faq_dict.keys():
                            a = faq_dict[key]
                            for k in range(len(vals)):
                                if vals[k] not in a:
                                    a.append(vals[k])
                                    faq_dict[key] = a
                        else:
                            faq_dict[key] = vals
            except:
                print('Skipped file: {}'.format(filename))
            
            # Cleans blank lines and random sub-headings from the text (Part 2 of 2)
            new_lines = [line for line in new_lines if line not in faq_dict.keys() and line not in faq_dict.values() and 'FAQ' not in line]
            new_lines = [re.sub('H\d: ', '', line) for line in new_lines]
            new_lines = [re.sub('Paragraph: ', '', line) for line in new_lines]
            new_lines = [line for line in new_lines if line != '\n' and line != ' \n']
            final_lines = []
            final_lines = [line for line in new_lines if line not in final_lines]

            # Write cleaned text data to new text file
            if len(final_lines) > 0:
                f_cl = open('txt_files/Cleaned/cleaned_{}'.format(filename), 'w+')
                for i in range(len(final_lines)):
                    f_cl.write(final_lines[i])
                f_cl.close()
        f.close()
    counter += 1
    print('Complete {0} {1}'.format(counter, filename))

Complete 1 advanced-python.txt
Complete 2 advanced-visualisation.txt
Complete 3 analytics-translation.txt
Complete 4 aws-cloud-practitioner.txt
Complete 5 blog.txt
Complete 6 blog_single.txt
Complete 7 build-pipeline.txt
Complete 8 contact.txt
Complete 9 data-analytics.txt
Complete 10 data-engineering.txt
Complete 11 data-science-for-executives.txt
Complete 12 data-science-for-managers.txt
Complete 13 data-science.txt
Complete 14 datascience-for-highschool.txt
Complete 15 data_explore.txt
Complete 16 deep-learning.txt
Complete 17 explore-data-science-academy.txt
Complete 18 explore-datascience.net.txt
Complete 19 find-your-course.txt
Complete 20 hire-explorer.txt
Complete 21 hire-talent.txt
Complete 22 home.txt
Complete 23 machine-learning.txt
Complete 24 money_back_promise.txt
Complete 25 our-careers.txt
Complete 26 our-courses.txt
Complete 27 our-explorers#Faculty.txt
Complete 28 our-explorers.txt
Complete 29 our-values.txt
Complete 30 payment-plans.txt
Complete 31 practice-areas.txt

In [4]:
for key in faq_dict.keys():
    print('\n')
    print(key)
    print(faq_dict[key])



H2: Why Python?

['H3: Master one of the most sought after skills of the future\n', 'Paragraph: Programming is important for speeding up processes as well as to automate, collect, manage, calculate, analyse processing of data and information accurately. People should program because it develops structured and creative thinking, and teaches persistence.\n', 'Paragraph: Python is one of the best languages to learn for the following reasons:\n', 'One of the easiest programming languages to learn.\n', 'It’s open source with a lot of documentation and online help.\n', 'It’s used by many big companies such as Google and Netflix.\n', 'It’s free.\n']


How to apply?

['All applications happen via our website: www.explore-datascience.net. Select the course and location (on campus or online) which works for you and click Apply Now. For on campus study, there are various requirements, for online applications it is simpler.\n', 'All applications happen via this website:. to apply for this course

In [5]:
# Manual checks
# Deleted files:
### 1. faq#Pricingcollapse3.txt and cleaned_faq#Pricingcollapse3.txt: No useful info
### 2. faq.txt and cleaned_faq.txt: No useful info

# Other info:
### 1. Many of the text files have info that also appears in the faq_dict. This info was not deleted because
###    it was recommended to leave as much informative text data in the files as possible.
### 2. contact.txt required email addresses. These were manually added.
### 3. Removed duplicated text from individual files. Some info is duplicated across files, this was not removed.
### 4. cleaned_data-engineering.txt: email address required on line 118, could not find it on website.



